# Prev

## include

In [111]:
include("../../src/struct_data.jl")
include("../../src/neighbor.jl")
include("../../src/forces/forces.jl")
include("../../src/run_event.jl")

RunFusionAggregates (generic function with 1 method)

## model

In [112]:
@time model = ModelSet(
    TimeModel(
        tₛᵢₘ  = 150000.0,
        dt    = 0.5,
        nₖₙₙ  = 100,
        nₛₐᵥₑ = 50
    ),
    InputModel(
        outer_ratio = 0.8,
        path_input  = "../../data/init/Sphere"
    ),
    OutputModel(
        name_output = "Test_1",
        path_output = ""
    ) 
)

ModelSet(TimeModel(150000.0, 0.5, 100, 50), InputModel(0.8, "../../data/init/Sphere"), OutputModel("Test_1", ""))

# Run

## Some Founded Parameters

### Cubic

In [113]:
# # Complete (Oriola Contractile Model) -> Normal Vectors of distance between cells without area
# Par1, Par2 = Cubic(0.0055,2.0,4.5), ContractilePar(0.162);
# Par1, Par2 = Cubic(0.0055,2.0,4.5), ContractilePar(0.166);
# Par1, Par2 = Cubic(0.0055,2.0,4.5), ContractilePar(0.164);

# # Complete (My model similar to Oriol -> Random Vector without area
Par1, Par2 = Cubic(0.02,2.0,4.0), ContractilePar(0.271);


## Fusion Agg

In [114]:
# Checking temp data for store
check_data("Test.xyz")
check_data("ErrorInit.xyz")

# Run Model
Par1, Par2 = Cubic(0.02,2.0,4.0), ContractilePar(0.271);
RunFusionAggregates(model::ModelSet, Par1, Par2, 15)

1×5008 CuArray{Int32, 2, CUDA.Mem.DeviceBuffer}:
 14  14  14  16  9  17  10  16  15  15  …  14  14  14  9  12  11  12  14  14

1000×5008 CuArray{Int32, 2, CUDA.Mem.DeviceBuffer}:
 1393   772  1312  3165   462  1510  …  2434  4932  3986  4998  2832  3920
  449  1409   285  1037    31   259     1177  3311  4501  4900  4432  4202
 1445  1218   496   680   641   825     4768  4353  4366  3507  3025  3605
  274   248   675  1047  1420   281     2094  4967  4084  4669  3214  4849
   84   828  2133   389   162    47     1693  4587  4979  4989  4361  4518
  806   197   148    85   213  1281  …  2086  4239  3709  4290  3678  4929
  469  3862   185   853    24  1237     4286  4397  4252  4786  3191  4978
 1573  1404    72  3950   596    11     1953  4499  4880  3089  4477  3720
 2762  3547   488   628     0   811     2391  3666  4151  4598  4369  4322
 1652  1641   710  2789     0   303        0  3604  4208  4096  4613  3698
    ⋮                             ⋮  ⋱                       ⋮        
    0     0     0     0     0     0        0     0     0     0     0     0
    0     0     0     0     0     0        0     0  

(19, CartesianIndex(1, 41))

# review

## mini test

In [115]:
# J , I = 10 , 5
# Neigbor = true

# check_data("miniTest.xyz")
# open("miniTest.xyz", "a") do f
#     write(f, "$(size(points, 1))\n")
#     write(f, "t=0\n")
#     writedlm(f,hcat(outline,Matrix(points)), ' ')
# end

# for j =1:J
#     for i = 1:I
#         # Compute the forces between each pair of particles in `agg` and their displacement.
#         threads=(16,3)
#         @cuda(
#             threads=threads,
#             blocks=(cld.(size(points,1)+1,threads[1]),1),
#             sum_force!(idx,random,idx_sum,points,force,ParForce,ParCont,dt,1)
#         )

#     end

#     if Neigbor == true
#         # Calculating Distance Matrix
#         threads=(64)
#         @cuda(
#             threads=threads,
#             blocks=cld.(size(points,1),threads),
#             dist_kernel!(idx,idx_cont,idx_sum,dist,points,r_max)
#         )
#     end

#     if j%200 == 1
#         prev_points = copy(points)
#         open("miniTest.xyz", "a") do f
#             write(f, "$(size(points, 1))\n")
#             write(f, "t=$(j*J)\n")
#             writedlm(f,hcat(outline,Matrix(points)), ' ')
#         end
#     end
# end

# val = 50
# println("BIG Val        = $(val)")
# println("Any BIG value? = $(any(points .> val))")
# println("Any Nan Value? = $(any(isnan.(points)))")
# display(points)
# display(force)